In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
data_dir = '/content/drive/MyDrive/main2'
batch_size = 32
img_height = 150
img_width = 150

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 17652 files belonging to 7 classes.
Using 14122 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 17652 files belonging to 7 classes.
Using 3530 files for validation.


In [ ]:
from tensorflow.keras import layers

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = 7

In [ ]:
RN50 = ResNet50(weights= 'imagenet', include_top=False, input_shape= (img_height,img_width,3))
input = RN50.output
input = layers.GlobalAveragePooling2D()(input)
# input = layers.Dropout(0.2)
fpred = layers.Dense(num_classes, activation = 'softmax')(input)
model = tf.keras.models.Model(inputs = RN50.input, outputs = fpred)

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
class CustomCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if(logs.get('val_accuracy')>0.965):
        print("\n 96% val_acc reached")
        self.model.stop_training = True

In [ ]:
!mkdir -p /content/drive/MyDrive/Model/RN50_1/cp

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/Model/RN50_1/cp/'

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=30,
  callbacks = [model_checkpoint_callback]
)

model.save('/content/drive/MyDrive/Model/RN50_1')

Epoch 1/30
442/442 [==============================] - 5126s 12s/step - loss: 1.3544 - accuracy: 0.5125 - val_loss: 1.0207 - val_accuracy: 0.6969
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Model/RN50_1/cp/assets
Epoch 2/30
442/442 [==============================] - 107s 241ms/step - loss: 0.4562 - accuracy: 0.8352 - val_loss: 0.6917 - val_accuracy: 0.7756
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Model/RN50_1/cp/assets
Epoch 3/30
442/442 [==============================] - 107s 241ms/step - loss: 0.2933 - accuracy: 0.8962 - val_loss: 0.7186 - val_accuracy: 0.7612
Epoch 4/30
442/442 [==============================] - 106s 241ms/step - loss: 0.1886 - accuracy: 0.9324 - val_loss: 0.6252 - val_accuracy: 0.7756
Epoch 5/30
442/442 [==============================] - 107s 241ms/step - loss: 0.1469 - accuracy: 0.9478 - val_loss: 1.1410 - val_accuracy: 0.7116
Epoch 6/30
442/442 [==============================] - 107s 242ms/step - loss: 0.1055 - accuracy: 0.9635 - val

In [ ]:
model.save('/content/drive/MyDrive/Model/RN50_1')